# 予測処理

## Library Import

In [1]:
# データの取り扱いに関するライブラリ
import numpy as np # 高速計算
import pandas as pd # 表データの扱い

import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 自身がファイルを格納したディレクトリを指定
ROOT_DIR = '../input/'
submit_file_path = ROOT_DIR + 'sample_submit.csv'
intermediate_path = '../output/intermediate_file/'
model_path = '../output/model/'
pred_path = '../output/pred/'

# スクリプトのバージョン指定
fe_ver = 3
training_ver = 5
submit_ver = 2

today = dt.datetime.today().strftime("%Y%m%d")

## File Import

In [3]:
test_df = pd.read_parquet(f'{intermediate_path}test_df_fe_v{fe_ver}.parquet')

## モデルの読み込み

In [4]:
import pickle

with open(f'{model_path}all_base_model_v{training_ver}.pkl', "rb") as f:
    all_base_model_dict = pickle.load(f)
with open(f'{model_path}house_base_model_v{training_ver}.pkl', "rb") as f:
    house_base_model_dict = pickle.load(f)

In [5]:
all_model = all_base_model_dict['all_model']
house_model = house_base_model_dict['house_model']

all_base_cols = all_base_model_dict['all_base_cols']
house_base_cols = house_base_model_dict['house_base_cols']

# house_cat_cols = house_base_model_dict['cat_cols']

## 予測

#### カテゴリ型へ変更

In [6]:
cat_cols = ['building_category', 'land_area_kind', 'walk_distance_bin', 'building_land_chimoku',
            'land_chisei','land_road_cond', 'access_zone', 'fireproof_x_structure', 'structure_group'
]

test_df[cat_cols] = test_df[cat_cols].astype('category')

In [7]:
# すべての category 列のリスト
obj_cols = test_df.select_dtypes(['object']).columns.tolist()

test_df[obj_cols] = test_df[obj_cols].astype('category')

#### データの分割

In [8]:
house_idx = test_df['building_category'] == 'house'
non_house_idx = ~house_idx

test_df_house = test_df[house_idx]
test_df_all = test_df[non_house_idx]

In [9]:
# --- 東京23区 ---
TOKYO_23 = [
    '千代田区', '中央区', '港区', '新宿区', '文京区', '台東区',
    '墨田区', '江東区', '品川区', '目黒区', '大田区', '世田谷区',
    '渋谷区', '中野区', '杉並区', '豊島区', '北区', '荒川区',
    '板橋区', '練馬区', '足立区', '葛飾区', '江戸川区'
]

# --- 政令指定都市 ---
SEIREI_CITIES = [
    '札幌市', '仙台市', 'さいたま市', '千葉市', '横浜市', '川崎市', '相模原市',
    '新潟市', '静岡市', '浜松市', '名古屋市',
    '京都市', '大阪市', '堺市', '神戸市',
    '岡山市', '広島市', '北九州市', '福岡市', '熊本市'
]

# --- 首都圏（都道府県） ---
CAPITAL_PREFS = ['東京都', '神奈川県', '埼玉県', '千葉県']

# --- 県庁所在地（市名のみ） ---
PREF_CAPITALS = [
    '札幌市','青森市','盛岡市','仙台市','秋田市','山形市','福島市',
    '水戸市','宇都宮市','前橋市','さいたま市','千葉市','新宿区',
    '横浜市','新潟市','富山市','金沢市','福井市','甲府市','長野市',
    '岐阜市','静岡市','名古屋市','津市','大津市','京都市','大阪市',
    '神戸市','奈良市','和歌山市','鳥取市','松江市','岡山市','広島市',
    '山口市','徳島市','高松市','松山市','高知市','福岡市','佐賀市',
    '長崎市','熊本市','大分市','宮崎市','鹿児島市','那覇市'
]

main_city = test_df_all.index[
    (
        (test_df_all['Prefecture name'] == '東京都') &
        (test_df_all['City/town/village name'].isin(TOKYO_23))
    )
    |
    (test_df_all['City/town/village name'].isin(['大阪市', '名古屋市']))
]

mid_city = test_df_all.index[
    (
        # 首都圏（23区除外）
        (
            test_df_all['Prefecture name'].isin(CAPITAL_PREFS)
            &
            ~(
                (test_df_all['Prefecture name'] == '東京都') &
                (test_df_all['City/town/village name'].isin(TOKYO_23))
            )
        )
        |
        # 政令指定都市
        (test_df_all['City/town/village name'].isin(SEIREI_CITIES))
        |
        # 県庁所在地
        (test_df_all['City/town/village name'].isin(PREF_CAPITALS))
    )
    &
    ~test_df_all.index.isin(main_city)
]

other = test_df_all.index[
    ~test_df_all.index.isin(main_city)
    &
    ~test_df_all.index.isin(mid_city)
]

urban_idx_dict = {
    'main_city': main_city,
    'mid_city': mid_city,
    'other': other,
}

In [10]:
idx_low_density = test_df_house.index[
    test_df_house['zone_residential_rank'] == 1
]

idx_mid_density = test_df_house.index[
    test_df_house['zone_residential_rank'] == 2
]

idx_high_density = test_df_house.index[
    test_df_house['zone_residential_rank'].isin([3, 4, 0]) |
    test_df_house['zone_residential_rank'].isna()
]

density_idx_dict = {
    'low': idx_low_density,
    'mid': idx_mid_density,
    'high': idx_high_density,
}

#### 関数

In [11]:
def _is_catboost_model(model: object) -> bool:
    """
    catboost が import されていない環境でも落ちないように判定する。
    """
    name = model.__class__.__name__.lower()
    mod = getattr(model.__class__, '__module__', '').lower()
    return ('catboost' in mod) or ('catboost' in name)


def _force_cat_cols_to_str(
    X: pd.DataFrame,
    cat_cols: list[str],
    na_token: str = 'NA',
) -> pd.DataFrame:
    """
    CatBoost の cat_features 用に、カテゴリ列を必ず str にし NaN を潰す。
    """
    if not cat_cols:
        return X

    X = X.copy()
    # base_colsに無いcat列が混ざっていても安全にする
    cat_cols_use = [c for c in cat_cols if c in X.columns]

    for c in cat_cols_use:
        # 何が入っていても「NA or str」に強制
        X[c] = X[c].map(lambda v: na_token if pd.isna(v) else str(v))

    return X


def predict_by_split(
    df: pd.DataFrame,
    models: dict[str, object],
    base_cols: list[str],
    idx_dict: dict[str, pd.Index],
    cat_cols: list[str] | None = None,   # ★追加
    na_token: str = 'NA',                # ★追加（任意）
) -> pd.Series:
    """
    split_key ごとに対応する model で予測し、df.index に揃えた Series を返す。
    CatBoost の場合のみ、cat_features 列を string 化して NaN / float を潰す。
    """
    pred = pd.Series(np.nan, index=df.index, dtype=float)
    cat_cols = cat_cols or []

    for split_key, model in models.items():
        idx = idx_dict.get(split_key)
        if idx is None:
            continue

        idx_use = pd.Index(idx).intersection(df.index)
        if len(idx_use) == 0:
            continue

        # ★ 必ず copy して、後続の型変換が元dfに波及しないようにする
        X = df.loc[idx_use, base_cols].copy()

        # ★ CatBoost のときだけカテゴリ列を強制変換
        if _is_catboost_model(model) and cat_cols:
            X = _force_cat_cols_to_str(X, cat_cols, na_token=na_token)

        pred_log = model.predict(X)
        pred.loc[idx_use] = np.exp(pred_log)

    return pred


#### house以外モデルの予測

In [12]:
pred_all = predict_by_split(
    df=test_df_all,
    models=all_model,
    base_cols=all_base_cols,
    idx_dict=urban_idx_dict
)

In [13]:
q = np.quantile(pred_all, [0, 0.25, 0.5, 0.75, 0.99, 1])
print("Min      :", q[0])
print("25% (Q1) :", q[1])
print("Median   :", q[2])
print("75% (Q3) :", q[3])
print("99%      :", q[4])
print("Max      :", q[5])

Min      : 4550261.420036911
25% (Q1) : 16553826.299107604
Median   : 25453675.26567359
75% (Q3) : 37538893.937677965
99%      : 99740588.05447176
Max      : 180151709.2044829


#### houseモデルの予測

In [14]:
pred_house = predict_by_split(
    df=test_df_house,
    models=house_model,
    base_cols=house_base_cols,
    idx_dict=density_idx_dict,
    # cat_cols=house_cat_cols
)

In [15]:
q = np.quantile(pred_house, [0, 0.25, 0.5, 0.75, 0.99, 1])
print("Min      :", q[0])
print("25% (Q1) :", q[1])
print("Median   :", q[2])
print("75% (Q3) :", q[3])
print("99%      :", q[4])
print("Max      :", q[5])

Min      : 4723983.534237512
25% (Q1) : 14031932.195870686
Median   : 21145631.810869895
75% (Q3) : 30303485.90278224
99%      : 93437986.00565428
Max      : 188604284.01164952


## 提出

In [16]:
test_pred_full = pd.Series(index=test_df.index, dtype=float)

test_pred_full.loc[non_house_idx] = pred_all
test_pred_full.loc[house_idx] = pred_house

In [17]:
submit_df = pd.read_csv(submit_file_path, header=None)
submit_df.columns = ['id', 'pred']

In [18]:
submit_df['pred'] = test_pred_full

In [19]:
submit_df.to_csv(
    f'{pred_path}submit_{today}_v{submit_ver}.csv',
    index=False,
    header=False
)